<h1><b>GSK NLP Analyst Technical Case Study</b><h1>

<h3>Christine C. Amuzie<h3>
Broader Impact Data Services

<i>Completed: 2021.12.10</i>

<i>Last Modified: 2021.12.10</i>

Select a drug from the dataset. Imagine that a leader for that product has asked you the following question: “What aspects do patients like about using our product, and what do people dislike?”
Use the data to write a report for this leader that will address their question.

In [1]:
# #!/Users/christineamuzie/opt/anaconda3/envs/p37env/bin/python
# CLI: 
# $ conda activate /Users/christineamuzie/opt/anaconda3/envs/p37env
# $ python --version


In [ ]:
import conda
# Set up installer
/Users/christineamuzie/opt/anaconda3/envs/p37env/bin/pip install 

In [6]:
sys.path.append("/Users/christineamuzie/opt/anaconda3/envs/p37env/bin/python")
import sys
print(sys.executable)
from subprocess import call
call("/Users/christineamuzie/opt/anaconda3/envs/p37env/bin/python /Users/christineamuzie/Documents/GitHub/Technical/GSK/analysis_1.0.ipynb", shell=True)


/Users/christineamuzie/opt/anaconda3/bin/python


1

In [18]:
import conda
conda activate py37env
# from conda import tensorflow
# Set up installer
# Users/christineamuzie/opt/anaconda3/condabin/conda install tensorflow

SyntaxError: invalid syntax (<ipython-input-18-038bb6ec700c>, line 2)

In [3]:
# Checking versions
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

# Loading relevant libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# Setting visualization parameters 
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

#Setting pytorch parameters
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.20.0

numpy       : 1.19.2
pandas      : 1.2.2
torch       : 1.10.0
transformers: 4.13.0



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Exploratory Data Analysis

df = pd.read_csv("case_study_data.txt", sep="\t", header=0)
df.head()
df.shape
df.info()
sns.countplot(df.drugName)
plt.xticks(rotation=90)
plt.xlabel('Review Counts by Drug');


We have more reviews for Phentermine than the other drugs, so we can proceed with the analysis of Phentermine reviews.

In [ ]:
# Data Preprocessing with BERT
PRE_TRAINED_MODEL_NAME = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
sample_txt = "It's a new dawn, it's a new day, it's a new life for me. And I'm feeling good!"
# Tokenization
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

tokenizer.sep_token, tokenizer.sep_token_id # sentence ending separation
tokenizer.cls_token, tokenizer.cls_token_id # beginning sentences
tokenizer.pad_token, tokenizer.pad_token_id # padding
tokenizer.unk_token, tokenizer.unk_token_id # unknown

encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=100,
  truncation=True, # explicitly truncate to max length
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  padding='max_length', # remove deprecated pad_to_max_length
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)
encoding.keys()

# dict_keys(['input_ids', 'attention_mask']) # dict_keys doesn't work for py3
vocab = list(df.keys())
print(vocab)

# https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/



In [ ]:
print(len(encoding['input_ids'][0])) # token ids
encoding['input_ids'][0]
print(len(encoding['attention_mask'][0])) # attention mask
encoding['attention_mask']
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]) # special tokens

In [ ]:
token_lens = [] # token length of each review
for txt in df.review:
  tokens = tokenizer.encode(txt, max_length=512, truncation=True)
  token_lens.append(len(tokens))

sns.distplot(token_lens) # plot distribution
plt.xlim([0, 512]);
plt.xlabel('Token count');

In [ ]:
MAX_LEN = 300 # majority of reviews < 300 tokens

In [ ]:
# determining max length of review
import numpy as np
measurer = np.vectorize(len)
res1 = measurer(df.values.astype(str)).max(axis=0)
print(res1)

In [ ]:
class RxReviewDataset(Dataset): # creating PyTorch dataset
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
df_train, df_test = train_test_split( # split data into training and test
  df,
  test_size=0.1,
  random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)
df_train.shape, df_val.shape, df_test.shape

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size): # create data loaders
  ds = RxReviewDataset(
    reviews=df.review.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

BATCH_SIZE = 16 # set batch size
MAX_LEN = 300 # majority of reviews < 300 tokens
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
class SentimentClassifier(nn.Module): # BERT-based classifier
  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = SentimentClassifier(len(class_names))
model = model.to(device) # move to GPU

In [ ]:
# initialize afinn sentiment analyzer
from afinn import Afinn
af = Afinn()

# compute sentiment scores (polarity) and labels
sentiment_scores = [af.score(article) for article in corpus]
sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]
    
    
# sentiment statistics per news category
df = pd.DataFrame([list(news_df['news_category']), sentiment_scores, sentiment_category]).T
df.columns = ['news_category', 'sentiment_score', 'sentiment_category']
df['sentiment_score'] = df.sentiment_score.astype('float')
df.groupby(by=['news_category']).describe()

In [ ]:
conda install pip

https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72

Broader Impact Data Services, LLC &copy; 2021